In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

thispath = Path.cwd().resolve()

datadir = Path(thispath.parent / "data")

csvdir = Path(datadir / "csv_folder/exatag_labels/outputs_csv/Lung/AOEC/Italian")

aoec_reports = pd.read_csv(csvdir / "gianmaria_silvello_Lung_AOEC_Italian.csv", index_col=0)

wsi_id = aoec_reports.index.values
labels_script = aoec_reports["label"].values

labels_automatic = pd.read_csv(datadir / "labels_v1.csv", index_col=0, dtype={"image_num": str})
labels_automatic_test = pd.read_csv(datadir / "labels_test_v1.csv", index_col=0)

labels_automatic_v2 = pd.read_csv(datadir / "labels.csv", index_col=0, dtype={"image_num": str})
labels_automatic_test_v2 = pd.read_csv(datadir / "labels_test.csv", index_col=0)

unique_labels = np.unique(np.array(labels_script))
print(unique_labels)

labels_report = {}
z=0
for id, label in zip(wsi_id, labels_script):
    if "_" in id:
        suffix = "0000"
        id = suffix + id[:-2]

    if id not in labels_report:
        if "No cancer" in label:
            labels_report[id] = [0, 0, 0, 1]
        elif "Cancer - small cell cancer" in label:
            labels_report[id] = [1, 0, 0, 0]
        elif "adenocarcinoma" in label:
            labels_report[id] = [0, 1, 0, 0]
        elif "squamous cell carcinoma" in label:
            labels_report[id] = [0, 0, 1, 0]
        elif "large" in label:
            z+=1
    else:
        if "No cancer" in label:
            labels_report[f"{id}_multi"] = [0, 0, 0, 1]
        elif "Cancer - small cell cancer" in label:
            labels_report[f"{id}_multi"] = [1, 0, 0, 0]
        elif "adenocarcinoma" in label:
            labels_report[f"{id}_multi"] = [0, 1, 0, 0]
        elif "squamous cell carcinoma" in label:
            labels_report[f"{id}_multi"] = [0, 0, 1, 0]
        elif "large" in label:
            z+=1

print(z)
temp = ""
multilabel_report = {}
for key, value in labels_report.items():
    if "multi" in key:
        multi_label = np.array(value) + np.array(labels_report[temp])
        multilabel_report[temp] = multi_label
    else:
        multilabel_report[key] = value
        temp = key
    


columns = ["cancer_scc", "cancer_nscc_adeno", "cancer_nscc_squamous", "no_cancer"]
multilabel_df = pd.DataFrame.from_dict(multilabel_report, orient='index', columns=columns)
multilabel_df.index.name = "image_num"
multilabel_df.sort_index(inplace=True)
multilabel_trval = multilabel_df.copy()
multilabel_test = multilabel_df.copy()


multilabel_trval["Set"] = multilabel_df.index.isin(labels_automatic.index)
multilabel_test["Set"] = multilabel_df.index.isin(labels_automatic_test.index)
multilabel_trval = multilabel_trval.loc[multilabel_trval["Set"]]
multilabel_test = multilabel_test.loc[multilabel_test["Set"]]
multilabel_trval.drop("Set", axis=1, inplace=True)
multilabel_test.drop("Set", axis=1, inplace=True)


multilabel_trval.to_csv(datadir / "manual_labels.csv")
multilabel_test.to_csv(datadir / "manual_labels_test.csv")

labels_automatic_as_manual = labels_automatic.copy()
labels_automatic_test_as_manual = labels_automatic_test.copy()
labels_automatic_as_manual["Set"] = labels_automatic_as_manual.index.isin(multilabel_trval.index)
labels_automatic_test_as_manual["Set"] = labels_automatic_test_as_manual.index.isin(multilabel_test.index)
labels_automatic_as_manual = labels_automatic_as_manual.loc[labels_automatic_as_manual["Set"]]
labels_automatic_test_as_manual = labels_automatic_test_as_manual.loc[labels_automatic_test_as_manual["Set"]]
labels_automatic_as_manual.drop("Set", axis=1, inplace=True)
labels_automatic_test_as_manual.drop("Set", axis=1, inplace=True)

labels_automatic_as_manual_v2 = labels_automatic_v2.copy()
labels_automatic_test_as_manual_v2 = labels_automatic_test_v2.copy()
labels_automatic_as_manual_v2["Set"] = labels_automatic_as_manual_v2.index.isin(multilabel_trval.index)
labels_automatic_test_as_manual_v2["Set"] = labels_automatic_test_as_manual_v2.index.isin(multilabel_test.index)
labels_automatic_as_manual_v2 = labels_automatic_as_manual_v2.loc[labels_automatic_as_manual_v2["Set"]]
labels_automatic_test_as_manual_v2 = labels_automatic_test_as_manual_v2.loc[labels_automatic_test_as_manual_v2["Set"]]
labels_automatic_as_manual_v2.drop("Set", axis=1, inplace=True)
labels_automatic_test_as_manual_v2.drop("Set", axis=1, inplace=True)


diff = labels_automatic_as_manual_v2.compare(multilabel_trval, align_axis=1)

print(diff)




['Cancer - non-small cell cancer, adenocarcinoma'
 'Cancer - non-small cell cancer, large cell carcinoma'
 'Cancer - non-small cell cancer, squamous cell carcinoma'
 'Cancer - small cell cancer' 'No cancer']
68
                   cancer_scc       cancer_nscc_adeno        \
                         self other              self other   
image_num                                                     
000029496800270408        NaN   NaN               1.0   0.0   
000029496800270409        NaN   NaN               1.0   0.0   
000029496800270410        NaN   NaN               1.0   0.0   
000029496800270411        NaN   NaN               1.0   0.0   
000029496800270412        NaN   NaN               1.0   0.0   
...                       ...   ...               ...   ...   
000035984200731450        NaN   NaN               1.0   0.0   
000036242800760961        NaN   NaN               1.0   0.0   
000036242800760967        NaN   NaN               1.0   0.0   
000036242800760969        NaN   N